https://adventofcode.com/2023/day/21

In [1]:
from collections import deque, defaultdict
from itertools import pairwise

import numpy as np
import numpy.linalg as la

In [2]:
with open("data/21.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
"""

In [4]:
def parse_puzzle(puzzle):
    paths = set()
    start = None
    for r, line in enumerate(puzzle.splitlines()):
        for c, char in enumerate(line):
            if char in ".S":
                pos = c - r * 1j
                paths.add(pos)
            if char == "S":
                start = pos
    return paths, start

In [5]:
def take_steps(paths, start, n):
    visited = {}
    q = deque([(start, 0)])
    while q:
        pos, step = q.popleft()
        if pos in visited:
            continue
        visited[pos] = step
        if step == n:
            continue
        nextstep = step+1
        for drxn in [1, 1j, -1, -1j]:
            nabe = pos + drxn
            if nabe in paths:
                q.append((nabe, nextstep)) 
    return sum(1 for stepcount in visited.values() if stepcount % 2 == n % 2)

In [6]:
paths, start = parse_puzzle(testdata)
take_steps(paths, start, 6)

16

In [7]:
%%time
paths, start = parse_puzzle(data)
take_steps(paths, start, 64)

CPU times: user 7.18 ms, sys: 0 ns, total: 7.18 ms
Wall time: 7.1 ms


3737

### Part 2

In [8]:
def parse_puzzle_2(puzzle):
    paths = set()
    start = None
    for r, line in enumerate(puzzle.splitlines()):
        for c, char in enumerate(line):
            if char in ".S":
                pos = c - r * 1j
                paths.add(pos)
            if char == "S":
                start = pos
    return paths, start, r+1, c+1

In [9]:
def take_more_steps(paths, start, rows, cols, n):
    visited = {}
    q = deque([(start, 0)])
    while q:
        pos, step = q.popleft()
        if pos in visited:
            continue
        visited[pos] = step
        if step == n:
            continue
        nextstep = step+1
        for drxn in [1, 1j, -1, -1j]:
            nabe = pos + drxn
            if cmod(nabe, rows, cols) in paths:
                q.append((nabe, nextstep)) 
    return sum(1 for stepcount in visited.values() if stepcount % 2 == n % 2)


def cmod(pos, rows, cols):
    # Negative coordinates and modulo are not a good fit
    x, y = int(pos.real), -int(pos.imag)
    return x % cols - (y % rows) * 1j


In [10]:
paths, start, rows, cols = parse_puzzle_2(data)
rows, cols

(131, 131)

In [11]:
%%time
L = []
for steps in range(1, 131*10+1, 131):
    L.append((steps, take_more_steps(paths, start, rows, cols, steps)))

CPU times: user 27 s, sys: 267 ms, total: 27.3 s
Wall time: 27.3 s


In [12]:
L

[(1, 4),
 (132, 15753),
 (263, 62064),
 (394, 138937),
 (525, 246372),
 (656, 384369),
 (787, 552928),
 (918, 752049),
 (1049, 981732),
 (1180, 1241977)]

In [13]:
d1 = [(a, b, d-b) for ((a, b), (c, d)) in zip(L, L[1:])]
d1

[(1, 4, 15749),
 (132, 15753, 46311),
 (263, 62064, 76873),
 (394, 138937, 107435),
 (525, 246372, 137997),
 (656, 384369, 168559),
 (787, 552928, 199121),
 (918, 752049, 229683),
 (1049, 981732, 260245)]

In [14]:
[(a, d, f-c) for ((a,b,c), (d,e,f)) in zip(d1, d1[1:])]

[(1, 132, 30562),
 (132, 263, 30562),
 (263, 394, 30562),
 (394, 525, 30562),
 (525, 656, 30562),
 (656, 787, 30562),
 (787, 918, 30562),
 (918, 1049, 30562)]

In [15]:
steps_period = 131
paths_period = 30562
final_steps = 26501365 

In [16]:
steps_offset = 26501365 % 131
steps_offset

65

In [17]:
s1 = steps_offset
s1p = take_more_steps(paths, start, rows, cols, s1)
s1p

3896

In [18]:
s2 = s1 + steps_period
s2p = take_more_steps(paths, start, rows, cols, s2)
s2p

34617

In [22]:
n = (final_steps - s2) // steps_period
n

202299

In [20]:
def triangle(n):
    return n * (n + 1) // 2

In [23]:
final_paths = s2p + (s2p - s1p) * n + paths_period * triangle(n)
final_paths

625382480005896